In [ ]:
import pandas
import matplotlib.pyplot as plt
import seaborn as sns


file_path: str = "../dataset"
files: tuple[str, ...] = ("connections", "devices", "processes", "profiles")

dataset = {}
for file in files:
    dataset[file] = pandas.read_csv(f"{file_path}/{file}.csv", sep="\t")
    

Connections description


In [ ]:
dataset["connections"].info()
summary = dataset["connections"].describe()
median = dataset["connections"].select_dtypes(include=["float64", "int64"]).median() # adding median to describe method output 
summary.loc["median"] = median 
summary

# MARKDOWN

In [ ]:
median.transpose().plot(kind="bar", figsize=(10, 6), width=0.8)

# Add a title and labels
plt.title("Summary Statistics with Median for Connections")
plt.xlabel("Columns")
plt.ylabel("Values")

# Rotate the x-axis labels for better readability
plt.xticks(rotation=45, ha="right")

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
dataset["connections"].head()

Devices description


In [ ]:
dataset["devices"].info()
dataset["devices"].describe()

In [ ]:
dataset["devices"].head()

Processes description


In [ ]:
dataset["processes"].info()
dataset["processes"].describe()

In [ ]:
dataset["processes"].head()

Profiles description


In [ ]:
dataset["profiles"].info()
dataset["profiles"].describe()

In [ ]:
dataset["profiles"].head()